# Demonstration Notebook: Parallel MBTI Inference Requests

This notebook demonstrates concurrent POST requests to a containerized FastAPI endpoint running a fine-tuned MBTI classification model hosted via Hugging Face Transformers.

IMPORTANT : The notebook assumes that the server is running locally on port 8000

In [1]:
import requests
import concurrent
import time

In [2]:
# endpoint URL for the prediction service
URL = "http://localhost:8000/predict"

In [3]:
# test texts to analyze
texts = [
    "Somebody mentioned not blaming personality type for lack of success or something... For me it has helped to know the limits of my type so I can stop wanting delusional things like being a pop star...",
    "Ugh, so much procrastination. I started a paper that I had multiple months to do the day that it was due at 3 a.m. and finished it at 6 a.m. even though it had to be 8 pages and be a synthesis of 5...",
    "Like being with someone who doesn't understand you and with whom you don't feel free to do be yourself. At least, I'd much rather be alone than with somebody who can't understand me.",
    "Ha, you sound like a cool guy. My life goals sound fairly similar to yours. I bet at this point you're going to click on my profile and be very disappointed to find that I'm not an INFP and I'm not a...",
    "Nope. You're not alone. I'm a singer and I love reading and writing fiction and listening to sad, deep music that I can get lost in and feel deep emotions with. I was a Humanities major in college as.",
    "When I lived in bigger cities, I solved this by going by myself or occasionally if friends wanted to join (although I prefer going with outgoing friends who can socialize independently) to networking",
    "Please, don't take photos of mundane, everyday moments that you'll forget anyway. In twenty years, it won't matter to you, or to the people you're with now. You'll even forget you have the photos.",
    "being a leader isn't necessarily always for a large group setting; you can be leader of only a few people. :] you don't have to picture yourself giving a lecture in front of hundreds of people, haha",
    "I really enjoy being alone these days",
    "I'm pretty outdoorsy. I like travelling with friends and family. I feel like I come alive when I'm surrounded by nature."
]

In [4]:
def send_request(text: str) -> dict:

    '''
    Sends a POST request to the prediction service with the given text.
    
    input: text (str): The text to be sent for prediction.
    output: dict: The response from the prediction service.

    '''

    try:
        response = requests.post(URL, json={"text": text})
        return response.json()
    
    except Exception as e:
        return {"error": str(e)}

In [5]:
def print_results(texts: list) -> None:
    
    '''
    Prints the results of the predictions for each text. Calls the send_request function concurrently for each text.
    
    input: texts (list): List of texts to be analyzed.
    output: None

    how it works:
    - Uses a ThreadPoolExecutor to send requests concurrently.
    - Measures the total time taken for all requests.
    - Prints the response for each text along with the total time taken.

    '''
    start_time = time.time()

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(send_request, texts))

    end_time = time.time()

    for i, res in enumerate(results):
        print(f"Text {i+1}: {texts[i]}")
        print(f"Response: {res}\n")

    print(f"Total time: {end_time - start_time:.2f} seconds")

In [7]:
print_results(texts)

Text 1: Somebody mentioned not blaming personality type for lack of success or something... For me it has helped to know the limits of my type so I can stop wanting delusional things like being a pop star...
Response: {'labels': ['Introvert', 'Intuition', 'Feeling', 'Perceiving'], 'mbti': 'INFP'}

Text 2: Ugh, so much procrastination. I started a paper that I had multiple months to do the day that it was due at 3 a.m. and finished it at 6 a.m. even though it had to be 8 pages and be a synthesis of 5...
Response: {'labels': ['Introvert', 'Intuition', 'Thinking', 'Perceiving'], 'mbti': 'INTP'}

Text 3: Like being with someone who doesn't understand you and with whom you don't feel free to do be yourself. At least, I'd much rather be alone than with somebody who can't understand me.
Response: {'labels': ['Introvert', 'Intuition', 'Feeling', 'Perceiving'], 'mbti': 'INFP'}

Text 4: Ha, you sound like a cool guy. My life goals sound fairly similar to yours. I bet at this point you're going t